## Obtención de dataset considerando información de septiembre y octubre

En este dataset vamos a recopilar las siguientes métricas:

- Cantidad de accesos en septiembre y octubre
- Entregas de actividades de evaluación continua que tuvieron lugar durante esos meses
- Máximo número de dias conectándose al aula
- Máximo número de días sin conectarse al aula

Muchas de ellas están ya calculadas, y algunas otras simplemente habrá que ajustarlas un poco a las nuevas fechas

## Configuración y carga de datasets previos

In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, lit, max
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import count
from pyspark.sql.functions import from_unixtime, date_format, to_date

import os

DATOS_LOG = "/home/carlos/Documentos/TFG/spark-workspace/data/raw/"
DATOS_LISTOS = "/home/carlos/Documentos/TFG/spark-workspace/data/datasets"
DATOS_DESTINO = "/home/carlos/Documentos/TFG/spark-workspace/data/datasets"

courseid_ip = 8683

# Crear sesión Spark
spark = SparkSession.builder \
    .appName("Creacion de metricas de septiembre y octubre") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "8") \
    .getOrCreate()

## Métricas del log

In [19]:
df_actividades = spark.read.parquet(f"{DATOS_LISTOS}/dataset_1.1.parquet")
df_log_listos = spark.read.parquet(f"{DATOS_LISTOS}/dataset_2.0.parquet")

print(f"Esquema de datos de actiivdades:\n")

df_actividades.printSchema()

print(f"Esquema de datos ya calculados del log:\n")

df_log_listos.printSchema()

# Cargar datos log raw
#==========================================================
df_log_ip = spark.read.parquet(f"{DATOS_LOG}/log_ip_cmi.parquet")
df_log_ip = df_log_ip.drop("courseid")

#Cargar datos alumnos
df_alumnos_ip = spark.read.parquet(f"{DATOS_LOG}/alumnos_ip_cmi.parquet")

#Quedarnos solo con las entradas de la ventana de observación
df_log_date = df_log_ip.withColumn("date", to_date(from_unixtime(col("timecreated"))))

df_log_obs = df_log_date.filter(
      (col("date") >= "2023-09-01") & (col("date") <= "2023-10-31")) \
.orderBy("date")

# Nos quedamos solo con los accesos al curso (No hace falta unir con alummnos matriculados ya que los logs estan calculados solo para ellos)
df_accesos_alumnos = df_log_obs.filter(
(col("component") == "core") & # Indicar el módulo de moodle que ha registrado la entrada, en este caso al tratarse de un acceso al aula virtual debe ser core
(col("action") == "viewed") & # Indicar la acción que se ha realizado, en este caso un acceso al aula virtual
(col("contextinstanceid") == courseid_ip) #Pasar el id del curso como contextinstanceid , dado que así indicamos que el acceso ha sido al curso y no a un recurso del mismo
)

print(f"Esquema de datos raw del log:\n")
df_accesos_alumnos.printSchema()



Esquema de datos de actiivdades:

root
 |-- userid: string (nullable = true)
 |-- Test Expr.: integer (nullable = true)
 |-- Test Complejidad: integer (nullable = true)
 |-- Act. 02 - Elecciones: integer (nullable = true)
 |-- Act. 03 - Catalan: integer (nullable = true)
 |-- Act. 04 - Primos: integer (nullable = true)
 |-- Act. 05 - Vectores: integer (nullable = true)
 |-- Act. 07: integer (nullable = true)
 |-- abandona: integer (nullable = true)
 |-- num_entregas: long (nullable = true)

Esquema de datos ya calculados del log:

root
 |-- userid: string (nullable = true)
 |-- nota_media_actividades: double (nullable = true)
 |-- proporcion_actividades_hechas: double (nullable = true)
 |-- num_accesos: long (nullable = true)
 |-- num_accesos_sept: long (nullable = true)
 |-- num_accesos_oct: long (nullable = true)
 |-- num_accesos_nov: long (nullable = true)
 |-- max_dias_sin_acceso: integer (nullable = true)
 |-- max_dias_consecutivos_accediendo: long (nullable = true)
 |-- abandona:

## Filtrado de datos


In [20]:
# Select specific columns from df_actividades
from pyspark.sql.functions import col

from pyspark.sql.functions import col

# Select specific columns using col() with backticks
df_actividades = df_actividades.select(
    col("userid"),
    col("`Test Expr.`"),
    col("`Act. 02 - Elecciones`"),
    col("`Act. 03 - Catalan`"),
    col("`Act. 04 - Primos`")
)
# Print the schema of the updated DataFrame
df_actividades.printSchema()

df_log_listos = df_log_listos.select("userid", "num_accesos_sept", "num_accesos_oct")

df_log_listos.printSchema()

df_sept_oct = df_log_listos.join(
      df_actividades, on="userid", how="left"
).select(
    col("userid"),
    col("`Test Expr.`"),
    col("`Act. 02 - Elecciones`"),
    col("`Act. 03 - Catalan`"),
    col("`Act. 04 - Primos`"),
    "num_accesos_sept",
    "num_accesos_oct"
)

df_sept_oct.printSchema()

display(df_sept_oct.toPandas().head(10))



root
 |-- userid: string (nullable = true)
 |-- Test Expr.: integer (nullable = true)
 |-- Act. 02 - Elecciones: integer (nullable = true)
 |-- Act. 03 - Catalan: integer (nullable = true)
 |-- Act. 04 - Primos: integer (nullable = true)

root
 |-- userid: string (nullable = true)
 |-- num_accesos_sept: long (nullable = true)
 |-- num_accesos_oct: long (nullable = true)

root
 |-- userid: string (nullable = true)
 |-- Test Expr.: integer (nullable = true)
 |-- Act. 02 - Elecciones: integer (nullable = true)
 |-- Act. 03 - Catalan: integer (nullable = true)
 |-- Act. 04 - Primos: integer (nullable = true)
 |-- num_accesos_sept: long (nullable = true)
 |-- num_accesos_oct: long (nullable = true)



,userid,Test Expr.,Act. 02 - Elecciones,Act. 03 - Catalan,Act. 04 - Primos,num_accesos_sept,num_accesos_oct
0,e1f1d0f48ca77093f9d66cefd325504245277db3e6c145...,1,1,1,1,63,81
1,b5de2bb5b8538b199d6b3f0ecb32daa8a9d730ccc484db...,1,1,1,1,53,58
2,90a634296aff946e9d045997d512d2b77dbc01880715c1...,1,1,1,1,31,40
3,b6b2a12e84ea8203775195ed2bb4e99c5788053782b0bd...,1,1,1,1,74,65
4,fd96e32a94a932f45eb32933d9ffeb71f4addf9153a76b...,1,1,1,0,23,57
5,ad2273914219245f3a1d76fa50e1e719d5342979b9bbca...,1,1,1,1,46,55
6,dd7af0da56a7f883acf1ca25d39672bd045b4178fe320c...,1,1,1,1,56,56
7,ef4bbb74d085499f21d5fc387fd346e55e9522b1f8c3eb...,0,0,0,0,9,25
8,f26e5bad97adcfb35a8324e08a7dc7c0546367a1ef9c9f...,0,1,0,0,74,30
9,ad8918581d3a671eeeea0ee79cb06af4e674d13de05886...,1,0,1,1,72,42


## Recalcular las de máximo número de días sobre nuevas fechas


In [ ]:
# df_accesos_alumnos.select(col("date")).distinct().orderBy("date").show(200, truncate=False)
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, datediff, lit, col, count, max, lag

# =============Métrica 1: Máximo numero de días sin acceder al aula================

# Paso 1: Crear ventana por alumno, ordenada por fecha de acceso
ventana = Window.partitionBy("userid").orderBy("date")

# Paso 2: Calcular diferencia de días entre fechas consecutivas
df_con_diff = df_accesos_alumnos.withColumn("fecha_anterior", lag("date").over(ventana))
df_con_diff = df_con_diff.withColumn("dias_sin_acceso", datediff("date", "fecha_anterior"))

# Paso 3: Obtener el máximo salto de días por alumno
dias_maximos = df_con_diff.groupBy("userid").agg(
    max("dias_sin_acceso").alias("max_dias_sin_acceso")
)

dias_maximos.show(5, truncate = False)

# =============Métrica 2: Máximo numero de días consecutivos accediendo al aula================

from pyspark.sql.window import Window

# 1) Mantener un único registro por día y alumno
df_dias_unicos = df_accesos_alumnos.select("userid", "date").distinct()

# 2) Para cada alumno, generar un identificador constante dentro de cada racha
ventana_orden = Window.partitionBy("userid").orderBy("date")

df_rachas = (
    df_dias_unicos
        .withColumn("idx", row_number().over(ventana_orden))
        .withColumn("dias_desde_epoch", datediff(col("date"), lit("1970-01-01")))
        .withColumn("grupo_racha", col("dias_desde_epoch") - col("idx"))
)

# 3) Contar la longitud de cada racha y quedarnos con la máxima
df_longitud_rachas = (
    df_rachas
        .groupBy("userid", "grupo_racha")
        .agg(count("*").alias("longitud_racha"))
)

dias_consecutivos_max = (
    df_longitud_rachas
        .groupBy("userid")
        .agg(max("longitud_racha").alias("max_dias_consecutivos_accediendo"))
)

dias_consecutivos_max.show(5, truncate=False)

dias_consecutivos = dias_consecutivos_max.join(
    dias_maximos, on="userid", how="left"
).select(
    "userid",
    "max_dias_consecutivos_accediendo",
    "max_dias_sin_acceso"
)

dias_consecutivos.printSchema()
display(dias_consecutivos.toPandas().head(10))



+----------------------------------------------------------------+-------------------+
|userid                                                          |max_dias_sin_acceso|
+----------------------------------------------------------------+-------------------+
|006b0e7bd07cec05e0952cb61c30893f6d30d7962f9efc99d0f041f6fadcc320|7                  |
|00ded60939d4949cc46e46e865b25d3f11756733cf946087710c61eda02729e1|4                  |
|05912200993a87a89df1a6ca9ac3d6493e2c4cc178760d8ee1da41033ac01b3e|5                  |
|073b1d0ee1d3857d50ea87087b25bbc6f5dbdbd2e94bcf52b89c48afa37e8c16|3                  |
|080b2c8b65e9d941f12e62b7d2b9fa22b669f06aeed07df5683fdf93a799204d|7                  |
+----------------------------------------------------------------+-------------------+
only showing top 5 rows
+----------------------------------------------------------------+--------------------------------+
|userid                                                          |max_dias_consecutivos_acced

,userid,max_dias_consecutivos_accediendo,max_dias_sin_acceso
0,006b0e7bd07cec05e0952cb61c30893f6d30d7962f9efc...,6,7
1,00ded60939d4949cc46e46e865b25d3f11756733cf9460...,11,4
2,05912200993a87a89df1a6ca9ac3d6493e2c4cc178760d...,5,5
3,073b1d0ee1d3857d50ea87087b25bbc6f5dbdbd2e94bcf...,13,3
4,080b2c8b65e9d941f12e62b7d2b9fa22b669f06aeed07d...,3,7
5,091af124e119a447c7f6594fb2f7c4fbb678f669966db0...,4,7
6,0a2e27fd5eb3547b064f5bcd8a26472c8802e19a5c158b...,5,5
7,11c0d56ee71665bfe766f1a57c333061cb34d747204264...,4,5
8,13a777f8c88ba748246449dfb45dcb8f76056a22f0b11e...,19,3
9,1416df0e4f8e87e449252eb090626d70cd44503423c5e0...,2,7


## Unir resultados con etiquetas y exportar


In [35]:
df_sept_oct = df_sept_oct.alias("sept_oct").join(
      dias_consecutivos.alias("dias"), on="userid", how="left"
).select(
      col("sept_oct.userid"),
      col("sept_oct.`Test Expr.`"),
      col("sept_oct.`Act. 02 - Elecciones`"),
      col("sept_oct.`Act. 03 - Catalan`"),
      col("sept_oct.`Act. 04 - Primos`"),
      col("sept_oct.num_accesos_sept"),
      col("sept_oct.num_accesos_oct"),
      col("dias.max_dias_consecutivos_accediendo"),
      col("dias.max_dias_sin_acceso")
)



df_etiquetas = spark.read.parquet("/home/carlos/Documentos/TFG/spark-workspace/data/datasets/etiquetas/etiquetas_abandono_entregas.parquet")

df_sept_oct = df_sept_oct.join(df_etiquetas, on="userid", how="left").fillna(0)

df_sept_oct.printSchema()
display(df_sept_oct.toPandas().head(10))
      
df_sept_oct_pd = df_sept_oct.toPandas()
df_sept_oct_pd.to_parquet(f"{DATOS_DESTINO}/dataset_4.0.parquet", index=False)



root
 |-- userid: string (nullable = true)
 |-- Test Expr.: integer (nullable = true)
 |-- Act. 02 - Elecciones: integer (nullable = true)
 |-- Act. 03 - Catalan: integer (nullable = true)
 |-- Act. 04 - Primos: integer (nullable = true)
 |-- num_accesos_sept: long (nullable = true)
 |-- num_accesos_oct: long (nullable = true)
 |-- max_dias_consecutivos_accediendo: long (nullable = true)
 |-- max_dias_sin_acceso: integer (nullable = true)
 |-- abandona: integer (nullable = true)



,userid,Test Expr.,Act. 02 - Elecciones,Act. 03 - Catalan,Act. 04 - Primos,num_accesos_sept,num_accesos_oct,max_dias_consecutivos_accediendo,max_dias_sin_acceso,abandona
0,e1f1d0f48ca77093f9d66cefd325504245277db3e6c145...,1,1,1,1,63,81,12,4,0
1,b5de2bb5b8538b199d6b3f0ecb32daa8a9d730ccc484db...,1,1,1,1,53,58,7,6,0
2,90a634296aff946e9d045997d512d2b77dbc01880715c1...,1,1,1,1,31,40,6,6,1
3,b6b2a12e84ea8203775195ed2bb4e99c5788053782b0bd...,1,1,1,1,74,65,7,4,0
4,fd96e32a94a932f45eb32933d9ffeb71f4addf9153a76b...,1,1,1,0,23,57,5,6,0
5,ad2273914219245f3a1d76fa50e1e719d5342979b9bbca...,1,1,1,1,46,55,5,5,0
6,dd7af0da56a7f883acf1ca25d39672bd045b4178fe320c...,1,1,1,1,56,56,6,4,0
7,ef4bbb74d085499f21d5fc387fd346e55e9522b1f8c3eb...,0,0,0,0,9,25,4,9,1
8,f26e5bad97adcfb35a8324e08a7dc7c0546367a1ef9c9f...,0,1,0,0,74,30,4,7,1
9,ad8918581d3a671eeeea0ee79cb06af4e674d13de05886...,1,0,1,1,72,42,4,5,0
